# IQ vs Success - Using Monte Carlo in Bivariate Distributions 

A colleague and I were discussing the importance of Intelligence as an contributor to success. Both intelligence and success can be defined using many metrics, but say we decide to use [IQ](https://en.wikipedia.org/wiki/Intelligence_quotient) as measure for intelligence. And on the job success as a way of measuring success. This discussion was definitely inspired by a Twitter thread by [Taleb](https://twitter.com/nntaleb) on IQ. 

We created an internal company quiz - **"Say you selected employees who had above average IQ. What is the probability that their performance is above average? The known correlation between IQ and performance in 0.5. And both performance and IQ are normally distributed. Compare this to the case when selection is random with respect to IQ."** 

In this note we will use Monte Carlo simulation to compute the above probability. 

## Joint Probability Distributions 

The most important idea to solve this problem is to understand the concept of a joint probability distribution. In this case specifically, we want to compute the joint Probability Distribution Function (PDF) of 'Success' and 'IQ'. To understand joint PDF lets take the simplest case of coin tosses. Say we have 2 coins and we toss them one after that other. Each flip is a Bernoulli trail with a Bernoulli distribution. However the join outcome has a different distribution as given below. The joint probability density function of the coin tosses defines probabilities for each pair of outcomes. 


|Outcome coin 1 | Outcome Coin 2|
|---------------|---------------|
|Head           |Head           |
|Head           |Tail           |
|Tail           |Head           |
|Tail           |Tail           |

Computing the joint PDF for 'Success' and 'IQ' is a bit more complex as these variables are continuous. But you can imagine a condition where you want to compute the probability of 'Success' being in a particular range and 'IQ' being in some other range. As an illustration you can have table like the below computed for 'Success' and 'IQ' (Additional note: Scales for both variables are arbitrary).

|'IQ'           | 'Success'     |
|---------------|---------------|
|> 30           |> 30           |
|< -30          |> 30           |
|....           |....           |
|....           |....           |

If we are able to generate the above table, we will be able to compute how successful a above average 'IQ' person will be. But before we can do that we have to understand the idea of correlation and/or covariance.  

## Correlation and Covariance

In the coin toss example above, the tosses are **independent**. Toss number two has no connection to toss number 1. Thus when we see a head as outcome 1, we are equally likely to get either head or tail as outcome 2. In this case the variables of 'Success' and 'IQ' are correlated. There are multiple studies on the relation between success and IQ. A quick Internet search will generate multiple results. Some studies show that the correlation is high and some show that it is low. Here lets stick to a somewhat high correlation of 0.5 Some studies have put the correlation to as low as 0.3. 

Next we assume that both 'Success' and 'IQ' are normally distributed. IQ is normally distributed by definition. Success may not. Many organizations use a normal curve to rate their employees. In the real world success may not be normally distributed. For our example here, we assume success to be normal as well. As both out variables are normally distributed, the joint probability distribution is know are **BiVariate Normal Distribution** or **BiVariate Gaussian Distribution**. 

## Bivariate Normal Distribution 

If X and Y are two normally distributed random variables. They the joint PDF for the bivariate normal distribution is give by - 

\begin{equation*}
      \frac{1}{2 \pi  \sigma_X \sigma_Y \sqrt{1-\rho^2}}
      \exp\left(
        -\frac{1}{2(1-\rho^2)}\left[
          \frac{(x-\mu_X)^2}{\sigma_X^2} +
          \frac{(y-\mu_Y)^2}{\sigma_Y^2} -
          \frac{2\rho(x-\mu_X)(y-\mu_Y)}{\sigma_X \sigma_Y}
        \right]
      \right)
\end{equation*}
Where $\rho$ is the coefficient of correlation. The derivation of equation is beyond the scope of this note. However, we will try to explore the properties of this distribution using some visualization in plotly.

# Creating a bivariate normal PDF plot in Python

In [ ]:
# imports
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np
# setup plotly credentials
plotly.tools.set_credentials_file(username='csaurav', api_key='')

In [ ]:
value_range = 2
xx = np.sort(np.random.normal(0, 1, 300))  # draw xx from a normal distribution 
yy = np.sort(np.random.normal(0, 1, 300))  # drwa yy from a normal distribution
x,y=np.meshgrid(xx, yy)
mu_x = np.mean(xx)
sd_x = np.std(xx)
mu_y = np.mean(yy)
sd_y = np.std(yy)
rho = 0.5  # the expected correlation

# the formula to generate the pdf given X, Y
z = ((np.exp(-(1 / (2 * (1 - rho ** 2)))*(((x - mu_x) / sd_x) ** 2 + ((y - mu_y) / sd_y) ** 2 
                                    - 2 * rho * (x - mu_x) * (y - mu_y)/(sd_x * sd_y))))
     / (2 * np.pi * sd_x * sd_y * np.sqrt(1 - rho ** 2)))

In [ ]:
init_notebook_mode(connected=True)
colorscale = [[0.0, 'rgb(20,29,67)'],
              [0.1, 'rgb(28,76,96)'],
              [0.2, 'rgb(16,125,121)'],
              [0.3, 'rgb(92,166,133)'],
              [0.4, 'rgb(182,202,175)'],
              [0.5, 'rgb(253,245,243)'],
              [0.6, 'rgb(230,183,162)'],
              [0.7, 'rgb(211,118,105)'],
              [0.8, 'rgb(174,63,95)'],
              [0.9, 'rgb(116,25,93)'],
              [1.0, 'rgb(51,13,53)']]
textz = [['IQ: '+'{:0.5f}'.format(x[i][j])+'<br>Success: '+'{:0.5f}'.format(y[i][j]) +
          '<br>z: '+'{:0.5f}'.format(z[i][j]) for j in range(z.shape[1])] for i in range(z.shape[0])]

trace1 = go.Surface(
    x=tuple(x),
    y=tuple(y),
    z=tuple(z),
    colorscale=colorscale,
    text=textz,
    hoverinfo='text',
)
axis = dict(
    showbackground=True,
    backgroundcolor="rgb(230, 230,230)",
    showgrid=False,
    zeroline=False,
    showline=False)

layout = go.Layout(title="Bi-variate Probabilty Distribution Function",
                   autosize=False,
                   width=700,
                   height=600,
                   scene=dict(xaxis=dict(axis, range=[-1 * value_range, value_range],
                                         title='IQ'),
                              yaxis=dict(
                                  axis, range=[-1 * value_range, value_range],
                              title='Success'),
                              zaxis=dict(
                                  axis, range=[np.min(z) - (np.max(z) - np.min(z)), np.max(z)],
                                  title='Probability P("Success", "IQ")'),
                              aspectratio=dict(x=1,
                                               y=1,
                                               z=0.95)
                             )
                   )
z_offset = (np.min(z) - (np.max(z) - np.min(z)))*np.ones(z.shape)
x_offset = -2*np.ones(z.shape)
y_offset = -2*np.ones(z.shape)


def proj_z(x, y, z): return z  # projection in the z-direction


colorsurfz = proj_z(x, y, z)


def proj_x(x, y, z): return x


colorsurfx = proj_z(x, y, z)


def proj_y(x, y, z): return y


colorsurfy = proj_z(x, y, z)

textx = [['Success: '+'{:0.5f}'.format(y[i][j])+'<br>z: '+'{:0.5f}'.format(z[i][j]) +
          '<br>IQ: '+'{:0.5f}'.format(x[i][j]) for j in range(z.shape[1])] for i in range(z.shape[0])]
texty = [['IQ: '+'{:0.5f}'.format(x[i][j])+'<br>z: '+'{:0.5f}'.format(z[i][j]) +
          '<br>Success: '+'{:0.5f}'.format(y[i][j]) for j in range(z.shape[1])] for i in range(z.shape[0])]

tracex = go.Surface(z=list(z),
                    x=list(x_offset),
                    y=list(y),
                    colorscale=colorscale,
                    showlegend=False,
                    showscale=False,
                    surfacecolor=colorsurfx,
                    text=textx,
                    hoverinfo='text'
                    )
tracey = go.Surface(z=list(z),
                    x=list(x),
                    y=list(y_offset),
                    colorscale=colorscale,
                    showlegend=False,
                    showscale=False,
                    surfacecolor=colorsurfy,
                    text=texty,
                    hoverinfo='text'
                    )
tracez = go.Surface(z=list(z_offset),
                    x=list(x),
                    y=list(y),
                    colorscale=colorscale,
                    showlegend=False,
                    showscale=False,
                    surfacecolor=colorsurfx,
                    text=textz,
                    hoverinfo='text'
                    )

data = [trace1, tracex, tracey, tracez]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
plotly.tools.set_credentials_file(username='csaurav', api_key='')

In [ ]:
py.plot(fig)

In [ ]:
mean = (0, 0)  # Mean for both success and IQ is zero
cov = [[1, .5], [.5, 1]]  # covarnance matrix with assumption of sd =1 (any other sd gives the same result)
x = np.random.multivariate_normal(mean, cov, 10000)   # we draw from both IQ and Success from a Multivariate 
count_both = 0
count_pos_iq = 0
for i in range(len(x)):
    if (x[i, 0] > 0):
        count_pos_iq += 1
        if (x[i, 1] > 0):
            count_both += 1
        p = p + 1
count_both/count_pos_iq   # ration of values where Succuess > 0, IQ >0 to those where IQ > 0